# <span style="color:#336699">simplecube</span>
<hr style="border:2px solid #0077b9;">

<div style="text-align: left;">
    <a href="https://doi.org/10.5281/zenodo.17186644"><img src="https://zenodo.org/badge/DOI/10.5281/zenodo.17186644.svg" align="center"/></a>
    <a href="https://nbviewer.jupyter.org/github/brazil-data-cube/code-gallery/"><img src="https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg" align="center"/></a>
    <a href="https://www.tidyverse.org/lifecycle/#stable"><img src="https://img.shields.io/badge/lifecycle-stable-green.svg" align="center"/></a>
    <a href="https://pypi.org/project/simplecube/"><img src="https://img.shields.io/pypi/v/simplecube" align="center"/></a>
</div>

<br/>

<div style="text-align: center;font-size: 90%;">
    Gabriel Sansigolo<sup><a href="https://orcid.org/0000-0003-0789-5858"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
    <br/><br/>
    Earth Observation and Geoinformatics Division, National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contact: <a href="mailto:gabriel.sansigolo@inpe.br">gabriel.sansigolo@inpe.br</a>
    <br/><br/>
    Last Update: Oct 8, 2025
</div>

<br/>

<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;">
<b>Abstract.</b> This Jupyter Notebook gives an overview on how to use the simplecube package to <br/>create creating <em>multidimensional arrays</em> from satellite imagery.
</div>

<br/>
<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;font-size: 75%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>This Jupyter Notebook is a supplement to the following paper:</b>
    <div style="margin-left: 10px; margin-right: 10px">
    Sansigolo, G.; Ferreira, K. R.; Queiroz, G. R.; Marujo, R. F. B.<a href="https://doi.org/10.5281/zenodo.17186644" target="_blank">simplecube: a python package for creating multidimensional arrays from satellite imagery
    </div>
</div>

In [1]:
from simplecube import simple_cube

In [2]:
stac_url = "https://data.inpe.br/bdc/stac/v1"

In [3]:
S2_cube=simple_cube(
    stac_url=stac_url,
    collection="S2-16D-2",
    start_date="2024-08-01",
    end_date="2025-07-31",
    bbox="-45.5006,-13.0066,-45.0172,-12.6062",
    bands=["B04","B08"]
)

S2_cube

Fetching... : 100%|██████████| 24/24 [00:00<00:00, 270.96 scenes/s]


<xarray.Dataset> Size: 2GB
Dimensions:      (band: 1, x: 5308, y: 4657, time: 24)
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 42kB 5.908e+06 5.908e+06 ... 5.961e+06 5.961e+06
  * y            (y) float64 37kB 9.918e+06 9.918e+06 ... 9.872e+06 9.872e+06
    spatial_ref  int64 8B 0
  * time         (time) datetime64[ns] 192B 2024-07-27 2024-08-12 ... 2025-07-28
Data variables:
    red          (time, band, y, x) int16 1GB 434 415 414 438 ... 673 677 650
    nir          (time, band, y, x) int16 1GB 1714 1542 1324 ... 1971 1940 1926

In [4]:
nir_band = S2_cube.nir
red_band = S2_cube.red

ndvi = (nir_band - red_band)/(nir_band + red_band)

S2_cube['ndvi'] = ndvi

In [5]:
import xarray as xr
from scipy.signal import savgol_filter

ndvi_smoothed = xr.apply_ufunc(
    savgol_filter,
    S2_cube['ndvi'],
    kwargs={'window_length': 8, 'polyorder': 3},
    input_core_dims=[['time']],  
    output_core_dims=[['time']],
    dask="parallelized",
    output_dtypes=[S2_cube['ndvi'].dtype] 
)

S2_cube['ndvi_smoothed'] = ndvi_smoothed
S2_cube

<xarray.Dataset> Size: 12GB
Dimensions:        (band: 1, x: 5308, y: 4657, time: 24)
Coordinates:
  * band           (band) int64 8B 1
  * x              (x) float64 42kB 5.908e+06 5.908e+06 ... 5.961e+06 5.961e+06
  * y              (y) float64 37kB 9.918e+06 9.918e+06 ... 9.872e+06 9.872e+06
    spatial_ref    int64 8B 0
  * time           (time) datetime64[ns] 192B 2024-07-27 ... 2025-07-28
Data variables:
    red            (time, band, y, x) int16 1GB 434 415 414 438 ... 673 677 650
    nir            (time, band, y, x) int16 1GB 1714 1542 1324 ... 1940 1926
    ndvi           (time, band, y, x) float64 5GB 0.5959 0.5759 ... 0.4953
    ndvi_smoothed  (band, y, x, time) float64 5GB 0.5563 0.6434 ... 0.5331